# Overview

This notebook is used for the development of the new orientation class that will function in place of FZfile.py and RotRep.

In [1]:
# import the functions that need to be migrated to the new classes

import os
import sys
import numpy as np

from scipy.linalg import polar

sys.path.append("../../")

In [2]:
from RotRep import Mat2EulerZXZ, Mat2EulerZXZVectorized 
from RotRep import EulerZXZ2Mat, EulerZXZ2MatVectorized

from RotRep import Misorien2FZ1

from RotRep import rod_from_quaternion
from RotRep import Orien2FZ
from RotRep import GetSymRotMat

# from FZfile import generate_random_rot_mat
# from FZfile import misorien

In [3]:
from hexomap.orientation import Eulers
from hexomap.orientation import Quaternion

The FZfile is somehow tied to pycuda, we have to come back for it later.

# Euler angles and (active) rotation matrix conversion

The previous implementation is done directly between the two, now we need to add quaternion to the fold.

In [3]:
EulerZXZ2Mat??

In [4]:
euler = [-1.26124505, -1.34263514, 0.39034601]
print(EulerZXZ2Mat(euler))

[[ 0.36369141  0.08331575  0.9277861 ]
 [-0.85460473  0.42614345  0.29673643]
 [-0.37064715 -0.90081088  0.22618675]]


In [5]:
from hexomap.orientation import Eulers

In [6]:
e = Eulers(*euler)
print(e)
print(e.as_matrix)

Eulers(phi1=-1.26124505, phi=-1.34263514, phi2=0.39034601, in_radians=True, order='zxz', convention='Bunge')
[[ 0.36369141  0.08331575  0.9277861 ]
 [-0.85460473  0.42614345  0.29673643]
 [-0.37064715 -0.90081088  0.22618675]]


In [7]:
# quick test

eulers = ((np.random.random(100*3)-0.5)*2*np.pi).reshape(100, 3)

for me in eulers:
    try:
        np.testing.assert_allclose(EulerZXZ2Mat(me), Eulers(*me).as_matrix)
    except:
        print("error")
        print(me)
        print(EulerZXZ2Mat(me))
        print(Eulers(*me).as_matrix)


In [8]:
EulerZXZ2MatVectorized??

In [12]:
N = 10000

eulers = ((np.random.random(N*3)-0.5)*2*np.pi).reshape(N, 3)

In [13]:
%%timeit

m_old = EulerZXZ2MatVectorized(eulers)

1.17 ms ± 10.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [14]:
%%timeit

m_new = Eulers.eulers_to_matrices(eulers)

1.2 ms ± 22.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [11]:
np.testing.assert_allclose(m_old, m_new)

In [12]:
Mat2EulerZXZ??

In [4]:
# original method
# euler -> matrix -> euler

euler = np.radians([0, 0, 32])  # special case, also need to make sure that the single rotation is the last one.
m = EulerZXZ2Mat(euler)

np.testing.assert_allclose(euler, Mat2EulerZXZ(m))

In [5]:
# new method

euler = Eulers(*np.radians([0, 0, 32]))

m = euler.as_matrix

print(Eulers.from_matrix(m))
print(euler)

Eulers(phi1=0.0, phi=0.0, phi2=0.5585053606381855, in_radians=True, order='zxz', convention='Bunge')
Eulers(phi1=0.0, phi=0.0, phi2=0.5585053606381855, in_radians=True, order='zxz', convention='Bunge')


Both methods are self consistent, now need to make sure that both can get the same Euler angles from a random rotation matrix

In [13]:
F = np.random.random(3*3).reshape(3,3)

R, V = polar(F)

print(R)
print(V)

[[ 0.1825966   0.97572875  0.12087964]
 [ 0.75865109 -0.21803073  0.61393088]
 [-0.62538548  0.02039622  0.78004935]]
[[0.15624347 0.26898015 0.30354132]
 [0.26898015 0.82666703 0.24860005]
 [0.30354132 0.24860005 1.2425504 ]]


In [14]:
euler_o = Mat2EulerZXZ(R)

euler_n = Eulers.from_matrix(R).as_array

print(np.array(euler_o)%(2*np.pi), euler_n)

[2.94718489 0.67605164 4.74499126] [2.94718489 0.67605164 4.74499126]


In [4]:
Mat2EulerZXZVectorized??

In [6]:
Fs = np.random.random(5*3*3).reshape(5,3,3)
Rs = np.array([polar(me)[0] for me in Fs])

eulers_o = Mat2EulerZXZVectorized(Rs)

print(eulers_o)

[[1.58458428 2.13042316 1.14246718]
 [2.02289707 1.9894906  1.22023214]
 [3.3984074  0.68974633 2.49756028]
 [4.0690013  0.78992427 6.1013702 ]
 [2.35568927 1.71723111 0.35105617]]


In [7]:
eulers_n = Eulers.matrices_to_eulers(Rs)

print(eulers_n)

[[1.58458428 2.13042316 1.14246718]
 [2.02289707 1.9894906  1.22023214]
 [3.3984074  0.68974633 2.49756028]
 [4.0690013  0.78992427 6.1013702 ]
 [2.35568927 1.71723111 0.35105617]]


In [8]:
# now batch testing

N = 10000
Fs = np.random.random(N*3*3).reshape(N,3,3)
Rs = np.array([polar(me)[0] for me in Fs])

In [9]:
%%timeit
eulers_o = Mat2EulerZXZVectorized(Rs)

2.01 ms ± 87.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
%%timeit
eulers_n = Eulers.matrices_to_eulers(Rs)

1.45 ms ± 8.63 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [11]:
np.testing.assert_allclose(eulers_o, eulers_n)

# Quaternion dev

In [4]:
euler = np.radians([35, 10, 0])

e = Eulers(*euler)
q = Quaternion.from_eulers(e)

print(e)
print(q)
print(np.degrees(q.as_eulers.as_array))

Eulers(phi1=0.6108652381980153, phi=0.17453292519943295, phi2=0.0, in_radians=True, order='zxz', convention='Bunge')
Quaternion(w=0.95008776981561, x=0.08312190921349344, y=0.0262082373043233, z=0.29956152315159634, normalized=True)
[3.50000000e+01 1.00000000e+01 2.28951055e-15]


# Misorientation related calculation

In [15]:
Misorien2FZ1??